In [ ]:
!pip install biopython==1.80

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 17.3 MB/s eta 0:00:00


In [ ]:
# @title Import Libraries
from Bio.SeqUtils.ProtParam import ProteinAnalysis
from Bio.SeqUtils import ProtParamData
from sklearn import svm
from sklearn.preprocessing import MinMaxScaler
import numpy as np
import pandas as pd
import itertools
from sklearn.metrics import confusion_matrix, accuracy_score, matthews_corrcoef, precision_score, recall_score, f1_score
import pickle, gzip

In [ ]:
# @title Definition of the constants
alphabet = list('GAVPLIMFWYSTCNQHDEKRX')
# Cycles for the CV
cycles = [([3,4,5],[2],[1]),([1,4,5],[3],[2]),([1,2,5],[4],[3]),([1,2,3],[5],[4]),([2,3,4],[1],[5])] #train,val,test
# Define the parameter grid for C and gamma
C = [1, 2, 3, 3.5, 4, 6, 8]
gamma = [0.5, 0.7, 1, 1.5, 2, 2.5, "scale"]
# Define the models to test
models = [['cc'], ['cc', 'hp'], ['cc', 'ah'], ['cc', 'tm'], ['cc', 'ch'], ['cc', 'hp', 'ah'], ['cc', 'hp', 'tm'], ['cc', 'hp', 'ch'], ['cc', 'ah', 'tm'], ['cc', 'ah', 'ch'], ['cc', 'tm', 'ch'], ['cc', 'hp', 'ah', 'tm'], ['cc', 'hp', 'ah', 'ch'], ['cc', 'hp', 'tm', 'ch'], ['cc', 'ah', 'tm', 'ch'], ['cc', 'hp', 'ah', 'tm', 'ch']]
# Best model in the binary classification
mm = [['cc','tm','ch']]

In [ ]:
# @title Upload the training files
# Specify the column names
col_pos_train = ['ID', 'Species', 'Kingdom', 'Length', 'Cleavage', 'Set']
col_neg_train = ['ID', 'Species', 'Kingdom', 'Length', 'Transmembrane', 'Set']

# Load the TSV file into a DataFrame
pos_train = pd.read_csv('pos-train.tsv', sep='\t', header=None, names=col_pos_train)
neg_train = pd.read_csv('neg-train.tsv', sep='\t', header=None, names=col_neg_train)

# Display the first few rows of the DataFrame
#print(pos_train.head())
#print(neg_train.head())

In [ ]:
# @title Feature Selection
# Composition over the first 22 residues
def extract_cc(sequence, alphabet):
  cc = np.zeros(21)
  for char in alphabet:
    cc[alphabet.index(char)] = sequence.count(char)
  cc=cc/len(sequence)
  #analysed_seq = ProteinAnalysis(sequence)
  #cc = analysed_seq.get_amino_acids_percent(ProtParamData.kd) #return a dictionary
  string =''
  for aa in cc:
    string += str(aa)+'\t'
  return string #vector of length 21

#Padding the sequence
def padding(sequence, ws):
  d = int(ws/2)
  sequence_with_padding = "X"*d + sequence + "X"*d
  return sequence_with_padding

# Hydrophobicity over first 40 residues, max and average, Kyte & Doolittle scale, WS = 5
def extract_hp(sequence, ws=5):
  analysed_seq_padding = ProteinAnalysis(padding(sequence, ws))
  kd_pos_with_padding = analysed_seq_padding.protein_scale(ProtParamData.kd, ws)
  return (np.max(kd_pos_with_padding), np.mean(kd_pos_with_padding))

# Charged residues: K-R-H over first 40 residues, ws=3, max and normalized position
def extract_ch(sequence, ws=3):
  max = 0
  scale = {'A':0, 'R':1, 'N':0, 'D':0, 'C':0, 'Q':0, 'E':0, 'G':0, 'H':1, 'I':0, 'L':0, 'K':1, 'M':0, 'F':0, 'P':0, 'S':0, 'T':0, 'W':0, 'Y':0, 'V':0, 'X':0}
  analysed_seq_padding = ProteinAnalysis(padding(sequence, ws))
  kd_pos_with_padding = analysed_seq_padding.protein_scale(scale, ws)
  max = np.max(kd_pos_with_padding)
  norm = 1-((kd_pos_with_padding.index(max)+1)/len(sequence))
  return (max, norm)

# Alpha-helix propensity over first 40 residues, max and average, chou & fasman scale, WS = 7
def extract_ah(sequence, ws=7):
  scale = {'A': 1.420, 'R': 0.980, 'N': 0.670, 'D': 1.010, 'C': 0.700, 'Q': 1.110, 'E': 1.510, 'G': 0.570, 'H': 1.000, 'I': 1.080, 'L': 1.210, 'K': 1.160, 'M': 1.450, 'F': 1.130, 'P': 0.570, 'S': 0.770, 'T': 0.830, 'W': 1.080, 'Y': 0.690, 'V': 1.060}
  analysed_seq_padding = ProteinAnalysis(padding(sequence, ws))
  kd_pos_with_padding = analysed_seq_padding.protein_scale(scale, ws)
  return (np.max(kd_pos_with_padding), np.mean(kd_pos_with_padding))

# Transmembrane over the first 40 residues, max and average, Zhao & London scale, WS = 7
def extract_tm(sequence, ws=7):
  scale = {'A': 0.380, 'R': -2.570, 'N': -1.620, 'D': -3.270, 'C': -0.300, 'Q': -1.840, 'E': -2.900, 'G': -0.190, 'H': -1.440, 'I': 1.970, 'L': 1.820, 'K': -3.460, 'M': 1.400, 'F': 1.980, 'P': -1.440, 'S': -0.530, 'T': -0.320, 'W': 1.530, 'Y': 0.490, 'V': 1.460}
  analysed_seq_padding = ProteinAnalysis(padding(sequence, ws))
  kd_pos_with_padding = analysed_seq_padding.protein_scale(scale, ws)
  return (np.max(kd_pos_with_padding), np.mean(kd_pos_with_padding))

def extract_features(sequence, alphabet):
  cc = extract_cc(sequence[:22], alphabet)
  hp_max, hp_mean = extract_hp(sequence)
  ch_max, ch_pos = extract_ch(sequence)
  ah_max, ah_mean = extract_ah(sequence)
  tm_max, tm_mean = extract_tm(sequence)
  return f'{cc}{hp_max}\t{hp_mean}\t{ah_max}\t{ah_mean}\t{tm_max}\t{tm_mean}\t{ch_max}\t{ch_pos}'
  #string to print in the file in the format 'cc'\t'hp'\t'ah'\t'tm'\t'ch'

In [ ]:
# @title Creation of the customized files
#Create a customize file for the positives
with open('positive.fasta', 'r') as read:
  with open('svm_pos.tsv', 'w') as write:
    for line in read:
      if '>' in line:
        line = line[1:].rstrip()
        query = pos_train.query("ID == @line")
        if not query.empty:
          line = read.readline()
          features = extract_features(line.rstrip()[:40], alphabet)
          print(query['ID'].iloc[0]+'\t'+str(query['Set'].iloc[0])+'\t2\t'+line.rstrip()[:40]+'\t'+features, file=write)
  write.close
read.close

#Create a customize file for the negatives
with open('negative.fasta', 'r') as read:
  with open('svm_neg.tsv', 'w') as write:
    for line in read:
      if '>' in line:
        line = line[1:].rstrip()
        query = neg_train.query("ID == @line")
        if not query.empty:
          line = read.readline()
          features = extract_features(line.rstrip()[:40], alphabet)
          if query['Transmembrane'].iloc[0] == False:
            print(query['ID'].iloc[0]+'\t'+str(query['Set'].iloc[0])+'\t0\t'+line.rstrip()[:40]+'\t'+features, file=write)
          else:
            print(query['ID'].iloc[0]+'\t'+str(query['Set'].iloc[0])+'\t1\t'+line.rstrip()[:40]+'\t'+features, file=write)
  write.close
read.close

Output streaming troncato alle ultime 5000 righe.


<function TextIOWrapper.close()>

In [ ]:
# @title Upload the dataframes
#Read the new files and concatenate them in a unique dataframe
col = ['ID','Set','Class','Sequence','G','A','V','P','L','I','M','F','W','Y','S','T','C','N','Q','H','D','E','K','R','X','hp_max','hp_mean','ah_max','ah_mean','tm_max','tm_mean','ch_max','ch_pos']

pos = pd.read_csv('svm_pos.tsv', sep='\t', header=None, names=col)
neg = pd.read_csv('svm_neg.tsv', sep='\t', header=None, names=col)

svm_df = pd.concat([pos, neg], axis=0)
print(svm_df.shape)
print(svm_df.head())
print(svm_df.tail())

(7899, 33)
       ID  Set  Class                                  Sequence         G  \
0  O43155    5      2  MGLQTTKWPSHGAFFLKSWLIISLGLYSQVSKLLACPSVC  0.090909   
1  O43866    4      2  MALLFSLILAICTRPGFLASPSGVRLVGGLHRCEGRVEVE  0.045455   
2  O94985    5      2  MLRRPAPALAPAARLLLAGLLCGGGVWAARVNKHKPWLEP  0.045455   
3  P02649    4      2  MKVLWAALLVTFLAGCQAKVEQAVETEPEPELRQQTEWQS  0.045455   
4  P02743    5      2  MNKPLLWISVLTSLLEAFAHTDLSGKVFVFPRESVTDHVN  0.000000   

          A         V         P         L         I  ...         R    X  \
0  0.045455  0.000000  0.045455  0.136364  0.090909  ...  0.000000  0.0   
1  0.136364  0.000000  0.090909  0.227273  0.090909  ...  0.045455  0.0   
2  0.272727  0.000000  0.136364  0.318182  0.000000  ...  0.136364  0.0   
3  0.181818  0.136364  0.000000  0.181818  0.000000  ...  0.000000  0.0   
4  0.090909  0.045455  0.045455  0.227273  0.045455  ...  0.000000  0.0   

   hp_max  hp_mean    ah_max   ah_mean    tm_max   tm_mean    ch_max  ch_po

In [ ]:
# @title Create the input dataframe
def extract_df(model,df):
  mapping = {
      'cc':[i for i in range(4,25)],
      'hp':[25,26],
      'ah':[27,28],
      'tm':[29,30],
      'ch':[31,32]
  }
  in_col = []
  for m in model:
    in_col.extend(mapping[m])
  input_df = df.iloc[:, in_col]
  return input_df

In [ ]:
# @title Compute the performance and the parameters for all the encoding combinations
# Initialize MinMaxScaler
scaler = MinMaxScaler()

for model in models:
  acc_mean, mcc_mean, precision_mean, recall_mean, f_mean = [], [], [], [], []
  C_temp, gamma_temp=[],[]
  for cycle in cycles: #cross-val
    df_train = svm_df[svm_df['Set'].isin(cycle[0])]
    y_train = df_train['Class'].to_numpy()
    x_train = extract_df(model,df_train)
    df_val = svm_df[svm_df['Set'].isin(cycle[1])]
    y_val = df_val['Class'].to_numpy()
    x_val = extract_df(model,df_val)
    df_test = svm_df[svm_df['Set'].isin(cycle[2])]
    y_test = df_test['Class'].to_numpy()
    x_test = extract_df(model,df_test)

    # Re-scaling of the features
    for col in range(21, x_train.shape[1]):
      column_name = x_train.columns[col]
      scaler.fit(x_train[[column_name]])
      x_train.loc[:, column_name] = scaler.transform(x_train[[column_name]]).flatten().astype('float64')
      x_val.loc[:, column_name] = scaler.transform(x_val[[column_name]]).flatten().astype('float64')
      x_test.loc[:, column_name] = scaler.transform(x_test[[column_name]]).flatten().astype('float64')

    f1_best = 0 #or mcc_best
    c_best, g_best = None, None
    for c in C:
      for g in gamma:
        #TRAINING
        # Create an instance of the SVC with RBF kernel
        my_svm = svm.SVC(C=c, kernel='rbf', probability=False, gamma=g)
        # Fit the model
        my_svm.fit(x_train, y_train)

        #VALIDATION
        y_pred = my_svm.predict(x_val)
        # Calculate the metric
        f1 = f1_score(y_val, y_pred, average='weighted')
        #mcc = matthews_corrcoef(y_val, y_pred)
        if (f1>f1_best) or (f1_best==None):
          c_best, g_best, f1_best = c, g, f1

    #TEST on c_best and g_best
    C_temp.append(c_best)
    gamma_temp.append(g_best)

    my_svm = svm.SVC(C=c_best, kernel='rbf', probability=False, gamma=g_best)
    my_svm.fit(x_train, y_train)
    y_pred_test = my_svm.predict(x_test)

    # Calculate metrics on the test
    accuracy = accuracy_score(y_test, y_pred_test)
    mcc = matthews_corrcoef(y_test, y_pred_test)
    precision = precision_score(y_test, y_pred_test, average='weighted')
    recall = recall_score(y_test, y_pred_test, average='weighted')
    f1 = f1_score(y_test, y_pred_test, average='weighted')

    acc_mean.append(accuracy)
    precision_mean.append(precision)
    recall_mean.append(recall)
    f_mean.append(f1)
    mcc_mean.append(mcc)

  # Select the best hyperparameters for that model from the cross-validation results
  # The best parameter is the one that was chosen more times during the CV
  num_c, num_g = 0,0
  tmp_c, tmp_g = 0,0
  par_c, par_g = 0,0
  for i in range(5):
    num_c = C_temp.count(C_temp[i])
    num_g = gamma_temp.count(gamma_temp[i])
    if (num_c==tmp_c) and (C_temp[i]<par_c):
      par_c, tmp_c = C_temp[i], num_c
    elif num_c>tmp_c:
      par_c, tmp_c = C_temp[i], num_c
    if (num_g==tmp_g):
      if par_g=='scale':
        par_g, tmp_g = gamma_temp[i], num_g
      if gamma_temp[i]!='scale':
        if (gamma_temp[i]<par_g):
          par_g, tmp_g = gamma_temp[i], num_g
    elif num_g>tmp_g:
      par_g, tmp_g = gamma_temp[i], num_g
  par = (par_c, par_g)

  # Store the CV performance in external file (with and without errors)
  with open('perf_errors_multiclass.tsv','a') as p_file:
    #model,c,gamma,Accuracy, precision, recall, f1, mcc
    print(f'{model}\t{par[0]}\t{par[1]}\t{np.mean(acc_mean):.3f} ± {(np.std(acc_mean))/np.sqrt(len(acc_mean)):.3f}\t{np.mean(precision_mean):.3f} ± {(np.std(precision_mean))/np.sqrt(len(precision_mean)):.3f}\t{np.mean(recall_mean):.3f} ± {(np.std(recall_mean))/np.sqrt(len(recall_mean)):.3f}\t{np.mean(f_mean):.3f} ± {(np.std(f_mean))/np.sqrt(len(f_mean)):.3f}\t{np.mean(mcc_mean):.3f} ± {(np.std(mcc_mean))/np.sqrt(len(mcc_mean)):.3f}', file=p_file)
  p_file.close

  with open('perf_multiclass.tsv','a') as p_file:
    #model,c,gamma,Accuracy, precision, recall, f1, mcc
    print(f'{model}\t{par[0]}\t{par[1]}\t{np.mean(acc_mean):.3f}\t{np.mean(precision_mean):.3f}\t{np.mean(recall_mean):.3f}\t{np.mean(f_mean):.3f}\t{np.mean(mcc_mean):.3f}', file=p_file)
  p_file.close

'''
  with open('perf_multiclass.tsv','a') as p_file:
    #model,c,gamma,Accuracy, precision, recall, f1, mcc
    print(f'{model}\t{par[0]}\t{par[1]}\t{np.mean(f_mean):.3f}\t{np.mean(mcc_mean):.3f}', file=p_file)
  p_file.close
'''


"\n  with open('perf_multiclass.tsv','a') as p_file:\n    #model,c,gamma,Accuracy, precision, recall, f1, mcc\n    print(f'{model}\t{par[0]}\t{par[1]}\t{np.mean(f_mean):.3f}\t{np.mean(mcc_mean):.3f}', file=p_file)\n  p_file.close\n"

In [ ]:
#col = ['Model','C','Gamma','accuracy','precision','recall','f1','mcc']
col = ['Model','C','Gamma','f1','mcc']
models_df = pd.read_csv('perf_multiclass.tsv', sep='\t', header=None, names=col)
print(models_df)

# @title Choose the final model and do its training
best_model_mcc = models_df.nlargest(1, 'mcc')

#using both mcc and f1 the full model is the best
c = best_model_mcc['C'].iloc[0]
g = float(best_model_mcc['Gamma'].iloc[0])

#train of the final model
best_model = ['cc', 'tm', 'ch']
y_final_train = svm_df['Class'].to_numpy()
x_final_train = extract_df(best_model,svm_df)
# Create an instance of the SVC with RBF kernel
final_svm = svm.SVC(C=c, kernel='rbf', probability=False, gamma=g)
# Fit the model
final_svm.fit(x_final_train, y_final_train)

# Save the model to file 'myModel.pkl.gz' using pickle
pickle.dump(final_svm, gzip.open('myModel_multiclass.pkl.gz', 'w'))

                             Model  C  Gamma     f1    mcc
0                           ['cc']  3  scale  0.847  0.618
1                     ['cc', 'hp']  8    2.5  0.870  0.666
2                     ['cc', 'ah']  8      2  0.845  0.627
3                     ['cc', 'tm']  8    2.5  0.888  0.705
4                     ['cc', 'ch']  6    0.5  0.844  0.620
5               ['cc', 'hp', 'ah']  8    2.5  0.872  0.666
6               ['cc', 'hp', 'tm']  8    2.5  0.889  0.707
7               ['cc', 'hp', 'ch']  8    2.5  0.872  0.667
8               ['cc', 'ah', 'tm']  8    2.5  0.888  0.707
9               ['cc', 'ah', 'ch']  6    0.5  0.844  0.621
10              ['cc', 'tm', 'ch']  8    1.5  0.891  0.713
11        ['cc', 'hp', 'ah', 'tm']  8    2.5  0.890  0.708
12        ['cc', 'hp', 'ah', 'ch']  8    2.5  0.873  0.666
13        ['cc', 'hp', 'tm', 'ch']  8    1.5  0.891  0.710
14        ['cc', 'ah', 'tm', 'ch']  8    1.5  0.891  0.710
15  ['cc', 'hp', 'ah', 'tm', 'ch']  6      1  0.888  0.7

### BENCHMARK TEST

In [ ]:
# @title Upload the benchmark files
# Specify the column names
col_pos_bench = ['ID', 'Species', 'Kingdom', 'Length', 'Cleavage']
col_neg_bench = ['ID', 'Species', 'Kingdom', 'Length', 'Transmembrane']

# Load the TSV file into a DataFrame
pos_bench = pd.read_csv('pos-bench.tsv', sep='\t', header=None, names=col_pos_bench)
neg_bench = pd.read_csv('neg-bench.tsv', sep='\t', header=None, names=col_neg_bench)

In [ ]:
# @title Creation of the customized files
# Create a customize file for the positives
with open('positive.fasta', 'r') as read:
  with open('svm_pos_test.tsv', 'w') as write:
    for line in read:
      if '>' in line:
        line = line[1:].rstrip()
        query = pos_bench.query("ID == @line")
        if not query.empty:
          line = read.readline()
          features = extract_features(line.rstrip()[:40], alphabet)
          # The final line will contain: id, class (2 for positive, 0 for negative, 1 for transmembrane), the first 40 residues and all the extracted features
          print(query['ID'].iloc[0]+'\t2\t'+line.rstrip()[:40]+'\t'+features, file=write)
  write.close
read.close

# Create a customize file for the negatives
with open('negative.fasta', 'r') as read:
  with open('svm_neg_test.tsv', 'w') as write:
    for line in read:
      if '>' in line:
        line = line[1:].rstrip()
        query = neg_bench.query("ID == @line")
        if not query.empty:
          line = read.readline()
          features = extract_features(line.rstrip()[:40], alphabet)
          # The final line will contain: id, class (2 for positive, 0 for negative, 1 for transmembrane), the first 40 residues and all the extracted features
          if query['Transmembrane'].iloc[0] == False:
            print(query['ID'].iloc[0]+'\t0\t'+line.rstrip()[:40]+'\t'+features, file=write)
          else:
            print(query['ID'].iloc[0]+'\t1\t'+line.rstrip()[:40]+'\t'+features, file=write)
  write.close
read.close

Output streaming troncato alle ultime 5000 righe.


<function TextIOWrapper.close()>

In [ ]:
# @title Upload the dataframes
# Read the new files and concatenate them in a unique dataframe
col_test = ['ID','Class','Sequence','G','A','V','P','L','I','M','F','W','Y','S','T','C','N','Q','H','D','E','K','R','X','hp_max','hp_mean','ah_max','ah_mean','tm_max','tm_mean','ch_max','ch_pos']

pos_test = pd.read_csv('svm_pos_test.tsv', sep='\t', header=None, names=col_test)
neg_test = pd.read_csv('svm_neg_test.tsv', sep='\t', header=None, names=col_test)

svm_test_df = pd.concat([pos_test, neg_test], axis=0)
print(svm_test_df.shape)

(1975, 32)


In [ ]:
# @title Create the input dataframe for the test
def extract_df_test(model,df):
  mapping = {
      'cc':[i for i in range(3,24)],
      'hp':[24,25],
      'ah':[26,27],
      'tm':[28,29],
      'ch':[30,31]
  }
  in_col = []
  for m in model:
    in_col.extend(mapping[m])
  input_df = df.iloc[:, in_col]
  return input_df

In [ ]:
# @title Test
y_bench = svm_test_df['Class'].to_numpy()
x_bench = extract_df_test(best_model,svm_test_df)

mySVC_from_file = pickle.load(gzip.open('myModel_multiclass.pkl.gz', 'r'))
y_pred_bench = mySVC_from_file.predict(x_bench)

# Calculate metrics on the benchmark
accuracy = accuracy_score(y_test, y_pred_test)
mcc = matthews_corrcoef(y_test, y_pred_test)
precision = precision_score(y_test, y_pred_test, average='weighted')
recall = recall_score(y_test, y_pred_test, average='weighted')
f1 = f1_score(y_test, y_pred_test, average='weighted')
cm = confusion_matrix(y_bench, y_pred_bench)

# Store the performance metrics in an external file
with open('svm_bench_res_multiclass.txt','w') as write:
  print("Confusion Matrix:\n"+str(cm), file=write)
  print('Accuracy: '+str(accuracy)+'\nPrecision: '+str(precision)+'\nRecall: '+str(recall)+'\nF1 Score: '+str(f1)+'\nMCC: '+str(mcc)+'\n', file=write)
write.close

<function TextIOWrapper.close()>

In [ ]:
print(cm)

[[1560    7   14]
 [ 101   58   18]
 [  17    6  194]]


In [ ]:
# Define the cells of the confusion metrics
from re import T
TN = cm[0][0]
TP = cm[2][2]
TT = cm[1][1]

FP = cm[0][2]
FN = cm[2][0]

N_FP_T = cm[0][1]
N_FN_T = cm[1][0]

P_FP_T = cm[1][2]
P_FN_T = cm[2][1]

# Tranform the 3x3 matrix in a 2x2 matrix to compare the results with the other models
# Taking into consideration that the transmembrane class is a subcategory of the negative class
tn = TN+TT+N_FN_T+N_FP_T
tp = TP
fn = FN+P_FN_T
fp = FP+P_FP_T

final_cm = np.array([[tn,fp],[fn,tp]])
print(final_cm)

[[1726   32]
 [  23  194]]


In [ ]:
# Calculate the performance metrics on the bi-dimensional matrix
accuracy = (tp + tn) / (tp + tn + fp + fn)
precision = tp / (tp + fp)
recall = tp / (tp + fn)
f1_score = 2 * (precision * recall) / (precision + recall)
numerator = (tp * tn) - (fp * fn)
denominator = np.sqrt((tp + fp) * (tp + fn) * (tn + fp) * (tn + fn))
mcc = numerator / denominator if denominator != 0 else 0


print("Confusion Matrix:\n"+str(cm))
print('Accuracy: '+str(accuracy)+'\nPrecision: '+str(precision)+'\nRecall: '+str(recall)+'\nF1 Score: '+str(f1)+'\nMCC: '+str(mcc)+'\n')

Confusion Matrix:
[[1560    7   14]
 [ 101   58   18]
 [  17    6  194]]
Accuracy: 0.9721518987341772
Precision: 0.8584070796460177
Recall: 0.8940092165898618
F1 Score: 0.8884324675188445
MCC: 0.8603961295295646



In [ ]:
# Save the results in an external file
with open('svm_bench_res_multiclass.txt','a') as write:
  print("Final Confusion Matrix:\n"+str(final_cm), file=write)
  print('Accuracy: '+str(accuracy)+'\nPrecision: '+str(precision)+'\nRecall: '+str(recall)+'\nF1 Score: '+str(f1)+'\nMCC: '+str(mcc)+'\n', file=write)
write.close

<function TextIOWrapper.close()>

## Analysis

In [ ]:
# Create the dataframe for the final analysis
svm_test_df['Prediction'] = y_pred_bench
bench = pd.concat([pos_bench, neg_bench], axis=0)
final_df = pd.merge(bench, svm_test_df, on='ID', how='outer')
# Convert the dataframe in a TSV
final_df.to_csv('svm_final_multiclass.tsv', sep='\t', index=False)

In [ ]:
svm = pd.read_csv('svm_final_multiclass.tsv', sep='\t')
print(svm.shape)
print(svm.head())

(1975, 38)
           ID                       Species        Kingdom  Length  Cleavage  \
0  A0A023W145         Ethmostigmus rubripes        Metazoa      88      24.0   
1  A0A0B5KU17    Penicillium brevicompactum          Fungi    2190       NaN   
2  A0A0F7YZQ7               Conus victoriae        Metazoa     100      24.0   
3  A0A0G2K047             Rattus norvegicus        Metazoa     683       NaN   
4  A0A0N7KIY3  Oryza sativa subsp. japonica  Viridiplantae     352       NaN   

  Transmembrane  Class                                  Sequence         G  \
0           NaN      2  MASQVVLSFALVVVLAVFVGQVDSCPSDCKCDYRSSQCRP  0.045455   
1         False      0  MNFHKGQPKEDLRVLFGPQCPDITDSITHIRDAISKDPTG  0.090909   
2           NaN      2  MAPSQKALLVLVLSMLLTASDSWARRIDCKVFVFAPICRG  0.000000   
3         False      0  MKPSWLQCRKVTGAGTLGAPLPGSPSVRGAGVARRALVAG  0.136364   
4         False      0  MEHHHLLLQLSPPPPPPPLPAAHLMMSPSFFDAGVFADVG  0.000000   

          A  ...    X  hp_max  hp_mean 

In [ ]:
print(cm)

[[1560    7   14]
 [ 101   58   18]
 [  17    6  194]]


In [ ]:
'''
TN = cm[0][0]
TP = cm[2][2]
TT = cm[1][1]

FP = cm[0][2]
FN = cm[2][0]

N_FP_T = cm[0][1]
N_FN_T = cm[1][0]

P_FP_T = cm[1][2]
P_FN_T = cm[2][1]

tn = TN+TT+N_FN_T+N_FP_T
tp = TP
fn = FN+P_FN_T
fp = FP+P_FP_T
'''

In [ ]:
# Calculate the False Positive Rates

#OPR = FP/Neg
OPR = fp/(fp+tn)

#tm_PR = FP(WithTransmembrane)/Neg(WithTransmembrane)
tm_PR = P_FP_T/(TT+N_FN_T+P_FP_T)

print(f'SVM method:\n Overall False Positive Rate: {OPR*100:.2f}%\n Transmembrane Positive Rate: {tm_PR*100:.2f}%')

# Save the results in an external file
with open('svm_bench_res_multiclass.txt','a') as write:
  print(f'SVM method:\n Overall False Positive Rate: {OPR*100:.2f}%\n Transmembrane Positive Rate: {tm_PR*100:.2f}%', file = write)
write.close

SVM method:
 Overall False Positive Rate: 1.82%
 Transmembrane Positive Rate: 10.17%


<function TextIOWrapper.close()>